In [0]:
# Instalando o lxml para processamento do HTML
%pip install lxml==6.0.1

In [0]:
# Importação das bibliotecas necessárias
from requests import get
from lxml import etree
from urllib.request import urlretrieve
from datetime import datetime, timedelta
from zipfile import ZipFile

myparser = etree.HTMLParser(encoding='utf-8')

In [0]:
def processamento_html(html):
    # Processando o HTML e buscando os arquivos zip
    tree = etree.HTML(html, parser=myparser)
    for link in tree.xpath('//table'):
        folders = [f for f in link.xpath('./tr/td/a/@href') if f.endswith('.zip')]
    return folders

def download_folders(url_base, mes, folders):
    # Realizando o download dos arquivos
    for folder in folders:
        url = f"{url_base}{mes}/{folder}"
        print(url)
        urlretrieve(url, f"{path_souce}/{folder}")

def unzip_files(path_souce: str, path_destination: str):
    paths = dbutils.fs.ls(path_souce)

    # Descompactando os arquivos
    for path in paths:
        with ZipFile(f"{path_souce}/{path.name}", 'r') as zipObj:
            zipObj.extractall(f"{path_destination}/{path.name.split('.')[0].lower()}")

        # Excluindo os arquivos compactados logo após descompactação
        dbutils.fs.rm(f"{path_souce}/{path.name}", True)

In [0]:

# Caminhos onde os arquivos serão baixados e descompactados
path_souce = '/Volumes/landing/raw/rfb_cnpj/temp'  # arquivos compactados zip
path_destination = '/Volumes/landing/raw/rfb_cnpj' # Arquivos descompactados
url_base = 'https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/'
mes = (datetime.today() - timedelta(days=10)).strftime('%Y-%m')

# Realizando a requisição
requisicao = get(f"{url_base}{mes}")

if requisicao.status_code != 200:
    print('Arquivos ainda não disponíveis')
else:
    html = requisicao.content
    folders = processamento_html(html)
    download_folders(url_base, mes, folders)
    unzip_files(path_souce, path_destination)
